In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
%sql
select * from datamodeling.silver.silvertable

order_id,order_date,customer_id,customer_name,customer_email,product_id,product_name,product_category,quantity,unit_price,payment_type,country,last_updated
1001,2024-07-01,1,Alice Johnson,alice@gmail.com,501,iPhone 14,Electronics,1,999.99,Credit Card,USA,2024-07-01
1002,2024-07-01,2,Bob Smith,bob@yahoo.com,502,AirPods Pro,Electronics,2,199.99,PayPal,USA,2024-07-01
1003,2024-07-01,3,Charlie Brown,charlie@outlook.com,503,Nike Shoes,Footwear,1,129.99,Credit Card,Canada,2024-07-01
1004,2024-07-02,4,David Lee,david@abc.com,504,Samsung S23,Electronics,1,899.99,Credit Card,USA,2024-07-02
1005,2024-07-02,1,Alice Johnson,alice@gmail.com,503,Nike Shoes,Footwear,2,129.99,Credit Card,USA,2024-07-02


## first we create dimension table and then fact table creation and fact tables have numbers,integers

## DIMCUSTOMER

In [0]:
%sql

create or replace table datamodeling.gold.goldcustomer
as
with removedup
as
(select distinct(customer_id),customer_name,customer_email from datamodeling.silver.silvertable)
select *, row_number() over(order by customer_id) as dimcustomerkey
from removedup


num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodeling.gold.goldcustomer

customer_id,customer_name,customer_email,dimcustomerkey
1,Alice Johnson,alice@gmail.com,1
2,Bob Smith,bob@yahoo.com,2
3,Charlie Brown,charlie@outlook.com,3
4,David Lee,david@abc.com,4


## DIMPRODUCT

In [0]:
%sql
create or replace table datamodeling.gold.goldproduct
as
with removedup
as
(select distinct(product_id),product_name,product_category from datamodeling.silver.silvertable)
select *, row_number() over(order by product_id) as dimproductkey
from removedup

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodeling.gold.goldproduct

product_id,product_name,product_category,dimproductkey
501,iPhone 14,Electronics,1
502,AirPods Pro,Electronics,2
503,Nike Shoes,Footwear,3
504,Samsung S23,Electronics,4


## DIMPAYMENT

In [0]:
%sql
create or replace table datamodeling.gold.goldpayment
as
with removedup
as
(select distinct(payment_type) from datamodeling.silver.silvertable)
select *, row_number() over(order by payment_type) as dimpaymentkey
from removedup

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodeling.gold.goldpayment

payment_type,dimpaymentkey
Credit Card,1
PayPal,2


## DIM COUNTRY

In [0]:
%sql
create or replace table datamodeling.gold.goldcountry
as
with removedup
as
(select distinct(country) from datamodeling.silver.silvertable)
select *, row_number() over(order by country) as dimcountrykey
from removedup

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodeling.gold.goldcountry

country,dimcountrykey
Canada,1
USA,2


## DIM SALES

In [0]:
%sql
create or replace table datamodeling.gold.goldsales
as
select 
  row_number() over(order by order_id) as dimsaleskey,
  order_id ,
  order_date ,
  customer_id ,
  customer_name ,
  customer_email ,
  product_id ,
  product_name ,
  product_category ,
  quantity ,
  unit_price ,
  payment_type ,
  country ,
  last_updated
  from datamodeling.silver.silvertable


num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodeling.gold.goldsales

dimsaleskey,order_id,order_date,customer_id,customer_name,customer_email,product_id,product_name,product_category,quantity,unit_price,payment_type,country,last_updated
1,1001,2024-07-01,1,Alice Johnson,alice@gmail.com,501,iPhone 14,Electronics,1,999.99,Credit Card,USA,2024-07-01
2,1002,2024-07-01,2,Bob Smith,bob@yahoo.com,502,AirPods Pro,Electronics,2,199.99,PayPal,USA,2024-07-01
3,1003,2024-07-01,3,Charlie Brown,charlie@outlook.com,503,Nike Shoes,Footwear,1,129.99,Credit Card,Canada,2024-07-01
4,1004,2024-07-02,4,David Lee,david@abc.com,504,Samsung S23,Electronics,1,899.99,Credit Card,USA,2024-07-02
5,1005,2024-07-02,1,Alice Johnson,alice@gmail.com,503,Nike Shoes,Footwear,2,129.99,Credit Card,USA,2024-07-02


## FACT TABLE

In [0]:
%sql
CREATE TABLE datamodeling.gold.facttable
as
SELECT 
C.dimcustomerkey,
P.dimproductkey,
PY.dimpaymentkey,
CO.dimcountrykey,
SA.dimsaleskey,
S.quantity,   
S.unit_price
FROM datamodeling.silver.silvertable as S
LEFT JOIN datamodeling.gold.goldcustomer as C
ON S.customer_id = C.customer_id
LEFT JOIN datamodeling.gold.goldproduct as P
ON S.product_id = P.product_id
LEFT JOIN datamodeling.gold.goldpayment as PY
ON S.payment_type = PY.payment_type
LEFT JOIN datamodeling.gold.goldcountry as CO
ON S.country = CO.country
LEFT JOIN datamodeling.gold.goldsales as SA
ON S.order_id = SA.order_id



num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodeling.gold.facttable

dimcustomerkey,dimproductkey,dimpaymentkey,dimcountrykey,dimsaleskey,quantity,unit_price
1,1,1,2,1,1,999.99
2,2,2,2,2,2,199.99
3,3,1,1,3,1,129.99
4,4,1,2,4,1,899.99
1,3,1,2,5,2,129.99
